!pip install demucs <br/>
!pip install yt_dlp<br/>
을 하면 오류창이 뜰 텐데 그러면 그냥 무시하고 다음코드 돌리면 돼유 <br/>
그리구 GPU로 설정하고 해야해유,,,ㅠ

In [1]:
!pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 

In [2]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.3 MB/s eta 0:00:00


In [8]:
from google.colab import drive
import numpy as np
import librosa
import pandas as pd
import os
import yt_dlp
import demucs.separate
import pickle
import shutil


drive.mount('/content/drive')
%cd /content/drive/MyDrive/demucs

mel_path = '/content/drive/MyDrive/demucs/mel_spectrogram/'
music_path = '/content/drive/MyDrive/music'

df = pd.read_csv("/content/drive/MyDrive/demucs/new_data.csv")

base_url = 'http://www.youtube.com/watch?v='
out_path = music_path
list_failed = []

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/demucs


In [4]:
def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')

def download_audio(url, name):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320'
        }],
        'outtmpl': f'{out_path}/{name}.%(ext)s',
        'noplaylist': True,
        'progress_hooks': [my_hook]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([base_url + url])
        except yt_dlp.utils.DownloadError:
            print(f"Download failed for ID: {url}")
            list_failed.append(url)

In [5]:
# Assume that your command is `demucs --mp3 --two-stems vocals -n mdx_extra "track with space.mp3"`
# The following codes are same as the command above:

#demucs.separate.main(["--mp3", "--two-stems", "vocals", "-n", "mdx_extra", "/content/drive/MyDrive/music/10cm.mp3"])

# # Or like this
# import demucs.separate
# import shlex
# demucs.separate.main(shlex.split('--mp3 --two-stems vocals -n mdx_extra "track with space.mp3"'))

In [21]:
for index, row in df[:1].iterrows():
    url = row['id']
    #name = f"{row['name']}_{row['song_title']}"
    name = row['name']+'_'+row['new_title'].strip()
    mp3name = (name+'.mp3').strip()
    download_audio(url, name)
    print(mp3name,' download finish')

    demucs.separate.main(["--mp3", "--two-stems", "vocals", "-n", "mdx_extra", music_path + '/' + mp3name])
    y, sr = librosa.load('/content/drive/MyDrive/demucs/separated/mdx_extra/'+ name +'/vocals.mp3', sr=16000)
    np.save(mel_path + name ,y)
    os.remove(music_path + '/' + mp3name)
    shutil.rmtree('/content/drive/MyDrive/demucs/separated/mdx_extra/'+ name)

with open("/content/drive/MyDrive/demucs/list_failed.pkl","wb") as f:
    pickle.dump(list_failed, f)

[youtube] Extracting URL: http://www.youtube.com/watch?v=kTVvtlfU-Ts
[youtube] kTVvtlfU-Ts: Downloading webpage
[youtube] kTVvtlfU-Ts: Downloading ios player API JSON
[youtube] kTVvtlfU-Ts: Downloading android player API JSON
[youtube] kTVvtlfU-Ts: Downloading m3u8 information
[info] kTVvtlfU-Ts: Downloading 1 format(s): 251
[download] Destination: /content/drive/MyDrive/music/윤하_우산 (Umbrella).webm
[download] 100% of    3.52MiB in 00:00:00 at 22.63MiB/s  Done downloading, now converting ...

[ExtractAudio] Destination: /content/drive/MyDrive/music/윤하_우산 (Umbrella).mp3
Deleting original file /content/drive/MyDrive/music/윤하_우산 (Umbrella).webm (pass -k to keep)
윤하_우산 (Umbrella).mp3  download finish
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/drive/MyDrive/demucs/separated/mdx_extra
Separating track /content/drive/MyDrive/music/윤하_우산 (Umbrella).mp3


100%|████████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:05<00:00, 40.99seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:05<00:00, 40.20seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:05<00:00, 39.76seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 231.0/231.0 [00:05<00:00, 41.38seconds/s]
